# Extracting from XML file

In [1]:
!git clone https://github.com/arpi-r/IR-Project.git

Cloning into 'IR-Project'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 72407 (delta 0), reused 3 (delta 0), pack-reused 72403
Receiving objects: 100% (72407/72407), 115.50 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (39829/39829), done.
Checking out files: 100% (48088/48088), done.


In [2]:
pip install Unidecode

     |████████████████████████████████| 245kB 12.5MB/s 


In [3]:
import os

import glob
import numpy as np
import nltk
import unidecode
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from prettytable import PrettyTable
from nltk.tokenize import word_tokenize
import xml.etree.cElementTree as ET
import csv
import json

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def remove_extra_space(text):
  words = text.strip().split()
  text = " ".join(words)
  return text

In [5]:
def parse_xml_file(filepath):
  tree = ET.parse(filepath)
  root = tree.getroot()
  trial_info = {}
  
  for child in root:
    if child.tag == 'brief_title' or child.tag == 'keyword':
      trial_info[child.tag] = child.text
    
    if child.tag == 'condition':
      if not child.tag in trial_info:
        trial_info[child.tag] = []
      trial_info[child.tag].append(child.text)
    
    if child.tag == 'intervention':
      for ch in child:
        if not ch.tag in trial_info:
          trial_info[ch.tag] = []
        trial_info[ch.tag].append(ch.text)
    
    if child.tag == 'brief_summary' or child.tag == 'detailed_description':
      trial_info[child.tag] = remove_extra_space(child.getchildren()[0].text)
    
    if child.tag == 'eligibility':
      for ch in child:
        if ch.tag == 'criteria':
          trial_info[ch.tag] = remove_extra_space(ch.getchildren()[0].text)
        else:
          trial_info[ch.tag] = ch.text      
    
    if child.tag == 'condition_browse' or child.tag == 'intervention_browse':
      for ch in child:
        if ch.tag == 'mesh_term':
          if not ch.tag in trial_info:
            trial_info[ch.tag] = []
          trial_info[ch.tag].append(ch.text)

  return trial_info

In [6]:
ls

IR-Project/  sample_data/


# Basic Data Pre-processing

In [7]:
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk.stem as st
from nltk import WordNetLemmatizer
import unidecode
import os

In [8]:
def preprocessData(text):
    # tokenize the text
    tokens = word_tokenize(text)

    # remove punctualtion and retain only alpha-numeric tokens
    tokens = [word for word in tokens if word.isalpha() or word.isdigit()]

    # normalize the tokens
    # lowercase
    tokens = [word.lower() for word in tokens]
    # remove accents
    tokens = [unidecode.unidecode(word) for word in tokens]
    # remove stopwords
    tokens = [word for word in tokens if not word in stopwords.words('english')]

    # lemmatize
    lemma = WordNetLemmatizer()
    tokens = [lemma.lemmatize(word, pos = 'v') for word in tokens]
    tokens = [lemma.lemmatize(word, pos = 'n') for word in tokens]
    return tokens
    

In [9]:
# vocabulary = set()
# for doc in os.listdir('Json_Data'):
#     with open(os.path.join('Json_Data', doc)) as json_file: 
#         try:
#             data = json.load(json_file)
#         except json.decoder.JSONDecodeError:
#             continue
#         for key in data:
#             if key == "id":
#                 continue
#             elif key == "gender" or key == "healthy_volunteers" or key == "intervention_type" or key == "intervention_name":
#                 vocabulary.update(data[key])
#                 continue
#             else:
#                 data[key] = preprocessData(str(data[key]))
#                 vocabulary.update(data[key])
#         out_file = open('./Preprocessed-Json/' + doc, "w") 
#         json.dump(data, out_file, indent = 6) 
#         out_file.close()
# with open('vocabulary.txt','w') as f:
#     f.write(str(vocabulary))

In [10]:
# !zip -r /content/Preprocessed-Json.zip /content/Preprocessed-Json

In [11]:
# from google.colab import files
# files.download("/content/Preprocessed-Json.zip")

In [12]:
# files.download("/content/vocabulary.txt")

# Access to preprocessed data

The above 2 steps have been run and the data is uploaded on GitHub. Their result can directly be accessed from this section. Only run cells containing imports and the preprocessData() method in the previous 2 sections.

In [13]:
# rm -rf ./IR-Project

In [14]:
# !git clone https://github.com/arpi-r/IR-Project.git

# Extract Test Queries

In [15]:
def parse_query_xml_file():
  filepath = '/content/IR-Project/queries/queries.xml'
  tree = ET.parse(filepath)
  root = tree.getroot()
  queries = []
  count = 1
  for child in root:
    query = {}
    unstructured_data = ""
    age = -1
    gender = ""
    other = ""
    mesh_terms = []

    for ch in child:
      if ch.tag == 'other':
        if not ch.text == 'None':
          unstructured_data += " " + ch.text
          other = ch.text

      if ch.tag == 'disease':
        unstructured_data += ch.text
        query[ch.tag] = ch.text

      if ch.tag == 'gene':
        unstructured_data += " " + ch.text
        query[ch.tag] = ch.text

      if ch.tag == 'demographic':
        info = ch.text.split(' ')
        age = int(info[0].split('-')[0])
        gender = info[1]
      
      if ch.tag == 'mesh_term':
        mesh_terms.append(ch.text)
    query['qid'] = count
    query['unstructured_data'] = unstructured_data
    query['age'] = age
    query['gender'] = gender
    query['other'] = other
    query['mesh_terms'] = mesh_terms
    count = count + 1

    queries.append(query)

  return queries

In [16]:
import random
def generateFold(queries, percent):
  train = []
  test = []
  visited = [False for i in range(len(queries))]
  while len(train) <= percent *  len(queries):
      ind = random.randint(0, len(queries) - 1)
      if not visited[ind]:
        train.append(queries[ind])
        visited[ind] = True
  
  for i in range(len(queries)):
    if not visited[i]:
      test.append(queries[i])
  return train, test
def generate_K_Folds(queries, percent, K):
  K_folds_Train = list()
  K_folds_Test = list()
  for i in range(K):
    train, test = generateFold(queries, percent)
    K_folds_Train.append(train)
    K_folds_Test.append(test)
  return K_folds_Train, K_folds_Test

# Exact match with structured data

In [17]:
def extract_structured_data_for_exact_match():
  path = '/content/IR-Project/Preprocessed-Json/'

  docs = {}
  for root, directories, files in os.walk(path, topdown=False):
    for name in files:
      filepath = (os.path.join(root,name))
      fileobj = open(filepath, 'r')
      file_data = json.load(fileobj)
      fileobj.close()
      if "xml" in file_data['id']:
        continue
      structured_data = []
      
      if 'minimum_age' in file_data:
        structured_data.append(file_data['minimum_age'])
      else:
        structured_data.append(0)

      if 'maximum_age' in file_data:
        structured_data.append(file_data['maximum_age'])
      else:
        structured_data.append(150)

      if 'gender' in file_data:
        structured_data.append(file_data['gender'])
      else:
        structured_data.append('all')

      if 'mesh_term' in file_data:
        structured_data.append(file_data['mesh_term'])
      else:
        structured_data.append([])
      
      docs[file_data['id']] = structured_data


  # print(len(docs))
  # print(docs['NCT00000108'])

  return docs

In [18]:
def get_relevant_set(query, docs):
  qage = query['age']
  qgen = query['gender']
  qmesh = query['mesh_terms']

  data_subset = set()
  l = len(docs)

  for key in docs.keys():
    doc = docs[key]
    # print(doc)
    age = True
    gender = True
    mesh_match = True

    if type(doc[0]) == int:
      if qage > doc[0] or qage < doc[1]:
        age = False
    else:
      if len(doc[0]) < 2 and len(doc[1]) < 2:
        age = True
      elif len(doc[0]) < 2:
        if not (doc[1][1] == 'year'):
          age = False
        if qage > int(doc[1][0]):
          age = False
      elif len(doc[1]) < 2:
        if not (doc[0][1] == 'year'):
          age = False
        if qage < int(doc[0][0]):
          age = False
      else:
        if qage > int(doc[1][0]) or qage < int(doc[0][0]):
          age = False

      if not (doc[2].lower() == 'all'):
        if not (qgen.lower() == doc[2].lower()):
          gender = False
      
      mesh = ' '.join(qmesh)
      mesh_terms = preprocessData(mesh)

      common_count = len(set(mesh_terms)&set(doc[3]))

      if common_count < 1:
        mesh_match = False
      
      if age and gender and mesh_match:
        data_subset.add(key)
  
  return data_subset

In [19]:
# docs = extract_structured_data_for_exact_match()
# query = queries[1]
# data_subset = get_relevant_set(query, docs)

# BM25 Model to extract relevant documents

In [20]:
!pip install rank_bm25

In [21]:
from rank_bm25 import BM25Okapi
import os
import json
import heapq 

In [22]:
def extract_unstructured_data_for_relevance_classification(data_subset):
  path = '/content/IR-Project/Preprocessed-Json/'

  corpus_data = []
  doc_id = []
  for root, directories, files in os.walk(path, topdown=False):
    for name in files:
      filepath = (os.path.join(root,name))
      fileobj = open(filepath, 'r')
      file_data = json.load(fileobj)
      fileobj.close()
      if "xml" in file_data['id'] or file_data['id'] not in data_subset:
        continue
      # print(file_data['id'])
      doc_id.append(file_data['id'])
      # summary = []
      unstructured_data = []
      if 'brief_title' in file_data:
        unstructured_data += file_data['brief_title']
      if 'brief_summary' in file_data:
        unstructured_data += file_data['brief_summary']
      # description = []
      if 'detailed_description' in file_data:
        unstructured_data += file_data['detailed_description']
      # unstructured_data = summary + description
      if 'criteria' in file_data:
        unstructured_data += file_data['criteria']
      if 'mesh_term' in file_data:
        unstructured_data += file_data['mesh_term']
      if 'keyword' in file_data:
        unstructured_data += file_data['keyword']
      # corpus_data.append(summary + description)
      corpus_data.append(unstructured_data)

  # print(len(corpus_data))
  # print(corpus_data[0])
  # print(len(corpus_data[0]))

  return doc_id, corpus_data

In [23]:
def get_bm25_model(data):
  bm25 = BM25Okapi(data)
  return bm25

In [24]:
def preprocess_query_bm25(query):
  tokenized_query = preprocessData(query)
  return tokenized_query

In [25]:
def find_relevancy(query, bm25, doc_id):
  preprocessed_query = preprocess_query_bm25(query)
  doc_scores = bm25.get_scores(preprocessed_query)

  # print("Doc scores: ", doc_scores)

  # ranked_heap = []
  # heapq.heapify(ranked_heap)

  l = len(doc_id)
  # for i in range(l):
  #   heapq.heappush(ranked_heap, (-doc_scores[i], doc_id[i]))

  relevant_docs = []
  irrelevant_docs = []

  irrelevant_count = 0
  for i in range(l):
    if doc_scores[i] == 0.0:
      irrelevant_count += 1
      irrelevant_docs.append(doc_id[i])
    else:
      relevant_docs.append(doc_id[i])
  relevant_count = l - irrelevant_count

  # print("Number of relevant docs: ", relevant_count)
  # print("Number of irrelevant docs: ", irrelevant_count)

  # doc_relevance_grade = []
  # count_per_grade = relevant_count // 3
  # count_so_far = 0
  # grade = 3

  # for i in range(0, relevant_count):
  #   doc = heapq.heappop(ranked_heap)[1]
  #   if (count_so_far > count_per_grade):
  #     count_so_far = 0
  #     grade -= 1
  #   count_so_far += 1
  #   doc_relevance_grade.append((doc, grade))

  # for i in range(0, irrelevant_count):
  #   doc = heapq.heappop(ranked_heap)[1]
  #   doc_relevance_grade.append((doc, 0))

  # print("Documents relevancy grade: ")
  # print(doc_relevance_grade)

  # for i in range(l):
  #   if doc_relevance_grade[i][1] == 0:
  #     irrelevant_docs.append(doc_relevance_grade[i][0])
  #   else:
  #     relevant_docs.append(doc_relevance_grade[i][0])
  
  # print("Relevant docs: ", relevant_docs)
  # print("Irrelevant docs: ", irrelevant_docs)

  # return doc_relevance_grade, relevant_docs, irrelevant_docs
  return relevant_docs, irrelevant_docs

In [26]:
# query = queries[1]['unstructured_data']
# doc_id, data_corpus = extract_unstructured_data_for_relevance_classification(data_subset)
# bm25 = get_bm25_model(data_corpus)
# print(query, bm25, doc_id)
# relevant_docs, irrelevant_docs = find_relevancy(query, bm25, doc_id)

# print(relevant_docs)
# print(irrelevant_docs)

# Query Reformulation - Rocchio's Method

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csc_matrix
from numpy import asarray
from numpy import dot
from numpy.linalg import norm

In [28]:
def extract_vsm_for_query_reformulation(query, data_subset):
  path = '/content/IR-Project/Preprocessed-Json/'
  corpus_data = []
  doc_ids = []

  for root, directories, files in os.walk(path, topdown=False):
    for name in files:
      filepath = (os.path.join(root,name))
      fileobj = open(filepath, 'r')
      file_data = json.load(fileobj)
      fileobj.close()
      if "xml" in file_data['id'] or file_data['id'] not in data_subset:
        continue
      # print(file_data['id'])
      doc_ids.append(file_data['id'])
      summary = []
      if 'brief_summary' in file_data:
        summary = file_data['brief_summary']
      description = []
      if 'detailed_description' in file_data:
        description = file_data['detailed_description']
      sum_des = summary + description
      s = ' '.join(sum_des)
      corpus_data.append(s)

  # print(corpus_data[0])
  corpus_data.append(query)
  
  vectorizer = TfidfVectorizer()
  tf = TfidfVectorizer(input=corpus_data, analyzer='word',
                      min_df = 0, stop_words = 'english', sublinear_tf=True)
  tfidf_matrix =  tf.fit_transform(corpus_data).toarray()

  data_vsm = tfidf_matrix[:-1][:]
  query_vsm = tfidf_matrix[-1][:]

  return tf, data_vsm, query_vsm, doc_ids

In [29]:
def preprocess_query_for_reformulation(query):
  preprocessed_query = preprocessData(query)
  preprocessed_query = ' '.join(preprocessed_query)
  # print(preprocessed_query)
  return preprocessed_query

In [30]:
def rocchios_query_reformulation(query, relevant, irrelevant, beta, gamma, docs, doc_ids):
  updated_query = query[:]
  l = len(doc_ids)
  rl = len(relevant)
  irl = len(irrelevant)
  if not (rl == 0):
    c1 = (beta / rl)
  else:
    c1 = 0
  if not (irl == 0):
    c2 = (gamma / irl)
  else:
    c2 = 0

  for i in range(l):
    if doc_ids[i] in relevant:
      updated_query += (c1 * docs[i])
    else:
      updated_query -= (c2 * docs[i])
    
    updated_query = updated_query.clip(0)

  return updated_query

In [31]:
def calc_similarity(docs_vsm, query_vsm):
  l = docs_vsm.shape[0]
  similarities = []

  for i in range(l):
    cos_sim = dot(docs_vsm[i], query_vsm)/(norm(docs_vsm[i])*norm(query_vsm))
    similarities.append(cos_sim)

  return similarities

In [32]:
def rank_cosine(similarities, doc_ids):
    ranked_heap = []
    heapq.heapify(ranked_heap)
    l = len(doc_ids)

    if not(len(similarities) == l):
      print("someting's wrong!")
      return ranked_heap

    for i in range(l):
      heapq.heappush(ranked_heap, (-similarities[i], doc_ids[i]))
    
    ranked_list = []

    for i in range(l):
      ranked_list.append(heapq.heappop(ranked_heap)[1])
    
    return ranked_list

In [33]:
# query = queries[1]['unstructured_data']
# preprocessed_query = preprocess_query_for_reformulation(query)
# tfidfvectorizer, docs_vsm, query_vsm, doc_ids = extract_vsm_for_query_reformulation(preprocessed_query, data_subset)
# print(docs_vsm.shape)
# print(query_vsm.shape)
# beta = 0.75
# gamma = 0.25
# runs = 10
# reformulated_query = query_vsm

# for i in range(runs):
#   reformulated_query = rocchios_query_reformulation(reformulated_query, relevant_docs, irrelevant_docs, beta, gamma, docs_vsm, doc_ids)

# # print("Original Query: ", query_vsm)
# # print("Reformulated Query: ", reformulated_query)
# print(sum(query_vsm))
# print(sum(reformulated_query))

# similarities = calc_similarity(docs_vsm, reformulated_query)
# print("\nSimilarities: ", similarities)

# ranked_list = rank_cosine(similarities, doc_ids)
# print("\nRanking: ", ranked_list)

In [34]:
# def query_reformulator(query, data_subset, beta, gamma, runs, relevant_docs, irrelevant_docs):
#   query = query['unstructured_data']
#   preprocessed_query = preprocess_query_for_reformulation(query)
#   tfidfvectorizer, docs_vsm, query_vsm, doc_ids = extract_vsm_for_query_reformulation(preprocessed_query, data_subset)
#   reformulated_query = query_vsm

#   for i in range(runs):
#     reformulated_query = rocchios_query_reformulation(reformulated_query, relevant_docs, irrelevant_docs, beta, gamma, docs_vsm, doc_ids)

#   similarities = calc_similarity(docs_vsm, reformulated_query)
#   ranked_list = rank_cosine(similarities, doc_ids)

#   return ranked_list


# def train_model(train_queries, docs, rocchio_iteration, beta, gamma) :
#   ranked_list = {}
#   for query in train_queries:
#     print(query)
#     data_subset = get_relevant_set(query, docs)
#     doc_id, data_corpus = extract_unstructured_data_for_relevance_classification(data_subset)
#     bm25 = get_bm25_model(data_corpus)
#     relevant_docs, irrelevant_docs = find_relevancy(query['unstructured_data'], bm25, doc_id)
#     ranked_list_after_rocchio = query_reformulator(query, data_subset, beta, gamma, rocchio_iteration, relevant_docs, irrelevant_docs)
#     ranked_list[query['qid']] = ranked_list_after_rocchio
#   train_LambdaRank(ranked_list)

# def main():
#   docs =  extract_structured_data_for_exact_match()
#   queries = parse_query_xml_file()
#   percent = 0.8
#   K = 10
#   r_itr = 10
#   beta = 0.75
#   gamma =  0.25
#   train, test = generate_K_Folds(queries, percent, K)
#   for i in range(len(train)):
#     train_model(train[i], docs, r_itr, beta, gamma)
#     break

# main()


# LamdaRank


In [35]:
import os
import json

def get_corpus():
  path = '/content/IR-Project/Preprocessed-Json/'
  corpus_data = {}
  doc_id = []

  for root, directories, files in os.walk(path, topdown=False):
    for name in files:
      filepath = (os.path.join(root,name))
      fileobj = open(filepath, 'r')
      file_data = json.load(fileobj)
      fileobj.close()
      if "xml" in file_data['id']:
        continue
      # print(file_data['id'])
      doc_id.append(file_data['id'])
      summary = []
      if 'brief_summary' in file_data:
        summary = file_data['brief_summary']
      description = []
      if 'detailed_description' in file_data:
        description = file_data['detailed_description']
      sum_des = summary + description
      s = ' '.join(sum_des)
      corpus_data[name[:len(name) - 5]] = s
  return corpus_data


In [36]:
!pip install LambdaRankNN
!pip install gensim

In [37]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def getDoc2Vec(corpus_data):
  documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus_data)]
  model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)
  w2v = {}
  for doc in corpus_data:
    w2v[doc] = model.infer_vector(corpus_data[doc])
  return model, w2v

In [38]:
import pandas as pd
def getRelevanceJudgement():
  relevancePath = '/content/IR-Project/queries/relevance.txt'

  data = pd.read_csv(relevancePath, header = None, delimiter = " ")
  relevance = {}
  qid = data[0]
  docid = data[2]
  relevanceJudgement = data[3]
  for i in range(len(docid)):
      relevance[(qid[i],docid[i])] = relevanceJudgement[i]
  return relevance


In [39]:
def getLambdaRankTrainData(temp_rankings, w2v, relevance):
  LambdaRank_X = []
  LambdaRank_Y = []
  LambdaRank_qid = []
  LambdaRank_docid = []
  for i in range(20):
    query = i + 1
    part_count = 10 
    temp_count = 0
    relgrade = 2
    for doc in temp_rankings[query]:
      if temp_count == part_count and not (relgrade == 0):
        relgrade -= 1
        temp_count = 0
        part_count = 20
      temp_count += 1
      LambdaRank_X.append(w2v[doc])
      LambdaRank_qid.append(query)
      LambdaRank_docid.append(doc)
      if (query, doc) in relevance:
        LambdaRank_Y.append(relevance[(query, doc)])
      else:
        LambdaRank_Y.append(relgrade)
  return  LambdaRank_X, LambdaRank_Y, LambdaRank_qid, LambdaRank_docid


In [40]:
def getLambdaRankTestData(temp_rankings, w2v, relevance):
  LambdaRank_X_test = []
  LambdaRank_Y_test = []
  LambdaRank_qid_test = []
  for i in range(20, 30):
    query = i + 1
    part_count = 10 
    temp_count = 0
    relgrade = 2
    for doc in temp_rankings[query]:
      if temp_count == part_count and not (relgrade == 0):
        relgrade -= 1
        temp_count = 0
        part_count = 20
      temp_count += 1
      LambdaRank_X_test.append(w2v[doc])
      LambdaRank_qid_test.append(query)
      if (query, doc) in relevance:
        LambdaRank_Y_test.append(relevance[(query, doc)])
      else:
        LambdaRank_Y_test.append(relgrade)
  return  LambdaRank_X_test, LambdaRank_Y_test, LambdaRank_qid_test

In [41]:
import numpy as np
import random
from LambdaRankNN import LambdaRankNN

# generate query data
def fitLambda(LambdaRank_X, LambdaRank_Y, LambdaRank_qid):
  X = np.array(LambdaRank_X)
  y = np.array(LambdaRank_Y)
  qid = np.array(LambdaRank_qid)
  ranker = LambdaRankNN(input_size=X.shape[1], hidden_layer_sizes=(16,8,), activation=('relu', 'relu',), solver='adam')
  ranker.fit(X, y, qid, epochs=5, batch_size = 64)
  return ranker

In [42]:
def train_LambdaRank(ranked_list, corpus_data): 
  d3v_model, w2v = getDoc2Vec(corpus_data)
  relevance = getRelevanceJudgement()
  x, y, qid, docid = getLambdaRankTrainData(ranked_list, w2v, relevance)
  x_test, y_test, qid_test = getLambdaRankTestData(ranked_list, w2v, relevance)
  ranker = fitLambda(x, y, qid)
  ranker.evaluate(np.array(x_test), np.array(y_test), np.array(qid_test), eval_at = 2)


def query_reformulator(query, data_subset, beta, gamma, runs, relevant_docs, irrelevant_docs):
  query = query['unstructured_data']
  preprocessed_query = preprocess_query_for_reformulation(query)
  tfidfvectorizer, docs_vsm, query_vsm, doc_ids = extract_vsm_for_query_reformulation(preprocessed_query, data_subset)
  reformulated_query = query_vsm

  for i in range(runs):
    reformulated_query = rocchios_query_reformulation(reformulated_query, relevant_docs, irrelevant_docs, beta, gamma, docs_vsm, doc_ids)

  similarities = calc_similarity(docs_vsm, reformulated_query)
  ranked_list = rank_cosine(similarities, doc_ids)

  return ranked_list


def train_model(train_queries, docs, rocchio_iteration, beta, gamma, fold, t) :
  ranked_list = {}
  for query in train_queries:
    data_subset = get_relevant_set(query, docs)
    doc_id, data_corpus = extract_unstructured_data_for_relevance_classification(data_subset)
    bm25 = get_bm25_model(data_corpus)
    relevant_docs, irrelevant_docs = find_relevancy(query['unstructured_data'], bm25, doc_id)
    ranked_list_after_rocchio = query_reformulator(query, data_subset, beta, gamma, rocchio_iteration, relevant_docs, irrelevant_docs)
    ranked_list[query['qid']] = ranked_list_after_rocchio

  json_object = json.dumps(ranked_list, indent = 4) 
  with open("ranked_list"+t+fold+".json", "w") as outfile: 
    outfile.write(json_object) 
  return ranked_list

import json
def getRankers(docs, queries, beta, gamma):
  rankers = []
  test_rank_list = []
  for r_itr in range(0,6):
    print("**************R_itr = ", r_itr,"**********************")
    ranked_list = train_model(queries, docs, r_itr, beta, gamma,"1","train")
    train_LambdaRank(ranked_list, get_corpus())
  #   rankers.append(ranker)
  # return rankers, test_rank_list

def main():
  docs =  extract_structured_data_for_exact_match()
  queries = parse_query_xml_file()
  score = []
  beta = 0.75
  gamma =  0.25
  getRankers(docs, queries, beta, gamma)
main()

**************R_itr =  0 **********************
Epoch 1/5
16767/16767 [==============================] - 23s 1ms/step - loss: 0.0082
Epoch 2/5
16767/16767 [==============================] - 23s 1ms/step - loss: 0.0052
Epoch 3/5
16767/16767 [==============================] - 23s 1ms/step - loss: 0.0040
Epoch 4/5
16767/16767 [==============================] - 23s 1ms/step - loss: 0.0032
Epoch 5/5
16767/16767 [==============================] - 23s 1ms/step - loss: 0.0026
ndcg: 0.6070941890124899
ndcg@2: 0.18175295903539446
**************R_itr =  1 **********************
Epoch 1/5
17157/17157 [==============================] - 23s 1ms/step - loss: 0.0074
Epoch 2/5
17157/17157 [==============================] - 24s 1ms/step - loss: 0.0031
Epoch 3/5
17157/17157 [==============================] - 24s 1ms/step - loss: 0.0021
Epoch 4/5
17157/17157 [==============================] - 24s 1ms/step - loss: 0.0017
Epoch 5/5
17157/17157 [==============================] - 26s 2ms/step - loss: 0.0015
n